In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Setting Matplotlib defaults
plt.style.use('seaborn-v0_8')
plt.rc('figure', figsize=(8,5), dpi=150)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=15, titlepad=10)
plt.rc('animation', html='html5')
plt.tight_layout()
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings
warnings.simplefilter('ignore')

pd.set_option('display.max_columns', 500)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e5/sample_submission.csv
/kaggle/input/playground-series-s5e5/train.csv
/kaggle/input/playground-series-s5e5/test.csv


<Figure size 1200x750 with 0 Axes>

In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s5e5/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/playground-series-s5e5/test.csv', index_col='id')

# Data Understanding

In [3]:
test.shape

(250000, 7)

In [4]:
train.head(10)

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
id,,,,,,,,
0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,female,38,166.0,61.0,25.0,102.0,40.6,146.0
5,female,26,156.0,56.0,19.0,100.0,40.5,103.0
6,female,21,172.0,73.0,3.0,81.0,38.3,9.0
7,male,46,188.0,94.0,23.0,100.0,40.8,145.0
8,female,33,166.0,63.0,25.0,107.0,40.5,161.0


In [5]:
train.describe()

,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
count,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000
mean,41.420404,174.697685,75.145668,15.421015,95.483995,40.036253,88.282781
std,15.175049,12.824496,13.982704,8.354095,9.449845,0.779875,62.395349
min,20.000000,126.000000,36.000000,1.000000,67.000000,37.100000,1.000000
25%,28.000000,164.000000,63.000000,8.000000,88.000000,39.600000,34.000000
50%,40.000000,174.000000,74.000000,15.000000,95.000000,40.300000,77.000000
75%,52.000000,185.000000,87.000000,23.000000,103.000000,40.700000,136.000000
max,79.000000,222.000000,132.000000,30.000000,128.000000,41.500000,314.000000


In [6]:
train.isnull().sum()

Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
Calories      0
dtype: int64

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 750000 entries, 0 to 749999
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Sex         750000 non-null  object 
 1   Age         750000 non-null  int64  
 2   Height      750000 non-null  float64
 3   Weight      750000 non-null  float64
 4   Duration    750000 non-null  float64
 5   Heart_Rate  750000 non-null  float64
 6   Body_Temp   750000 non-null  float64
 7   Calories    750000 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 51.5+ MB


# Data Preprocessing

## Reducing memory usage

In [8]:
train['Sex'] = train['Sex'].astype('category')
test['Sex'] = test['Sex'].astype('category')

In [9]:
train['Age'] = train['Age'].astype('int8')
test['Age'] = test['Age'].astype('int8')

In [10]:
num_cols = test.select_dtypes(include='float64').columns
for col in num_cols:
    train[num_cols] = train[num_cols].astype('float32')
    test[num_cols] = test[num_cols].astype('float32')

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 750000 entries, 0 to 749999
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   Sex         750000 non-null  category
 1   Age         750000 non-null  int8    
 2   Height      750000 non-null  float32 
 3   Weight      750000 non-null  float32 
 4   Duration    750000 non-null  float32 
 5   Heart_Rate  750000 non-null  float32 
 6   Body_Temp   750000 non-null  float32 
 7   Calories    750000 non-null  float64 
dtypes: category(1), float32(5), float64(1), int8(1)
memory usage: 27.2 MB


# Feature Understanding via Data Visualization

In [12]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train['Sex'] = le.fit_transform(train['Sex']).astype('int8')
test['Sex'] = le.transform(test['Sex']).astype('int8')

In [13]:
X = train.copy()
y = X.pop('Calories')
y = np.log1p(y)
X_test = test.copy()

In [14]:
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_log_error

n_folds = 4
kf = KFold(n_splits=n_folds, shuffle=True, random_state=100)

oof_dnn = np.zeros(len(y))
test_dnn = np.zeros(len(X_test))

In [15]:
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout, GaussianDropout
from tensorflow.keras.regularizers import l2

2025-05-06 09:44:53.657695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746524694.053381      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746524694.179687      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [16]:
from tensorflow.keras.optimizers import Adam, SGD, Nadam
from tensorflow.keras.metrics import RootMeanSquaredError

def build_model():
    return Sequential([
        Input(shape=(input_shape)),
        BatchNormalization(),
        
        Dense(280, activation='leaky_relu'),
        GaussianDropout(rate=0.2),
        BatchNormalization(),
        
        Dense(150, activation='leaky_relu'),
        GaussianDropout(rate=0.1),
        BatchNormalization(),
        
        Dense(300, activation='leaky_relu'),
        BatchNormalization(),
        GaussianDropout(rate=0.1),
        
        Dense(120, activation='leaky_relu'),
        BatchNormalization(),
        
        Dense(1,activation='linear'),
    ])
    

In [17]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

def make_callbacks():
    lr_callback = ReduceLROnPlateau(
            monitor='val_root_mean_squared_error',     
            factor=0.5,              
            patience=5,              
            verbose=1,               
            min_lr=1e-6            
        )
    
    early_stop = EarlyStopping(
        patience=15, 
        monitor='val_root_mean_squared_error', 
        restore_best_weights=True, 
        mode='min'
    )

    checkpoint = ModelCheckpoint(
        filepath="/kaggle/working/best_model.keras",
        monitor='val_root_mean_squared_error',
        save_best_only=True,
        mode='min',
        verbose=0
    )
    
    return [lr_callback, early_stop, checkpoint]

In [18]:
for fold, (train_idx, valid_idx) in enumerate(kf.split(X), start=1):
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

    X_test = test.copy()
    
    scaler = StandardScaler()
    X_train= scaler.fit_transform(X_train)
    X_valid = scaler.transform(X_valid)
    X_test = scaler.transform(X_test) 
    
    input_shape = (X_train.shape[1], )

    dnn = build_model()
    optimizer = Nadam(learning_rate=0.05)
    rmse = RootMeanSquaredError()
    
    dnn.compile(optimizer=optimizer, loss='mse', metrics=[rmse])
    
    dnn.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=250,
    epochs=25,
    callbacks=make_callbacks(),
    verbose=2
    )

    y_pred = dnn.predict(X_valid, batch_size=250, verbose=2).flatten()
    oof_dnn[valid_idx] = y_pred
    
    test_dnn += dnn.predict(X_test, batch_size=250, verbose=2).flatten()
    
    rmsle = np.sqrt(mean_squared_log_error(np.expm1(y_pred), np.expm1(y_valid)))
    print(f'Fold: {fold} RMSLE: {rmsle:,.6f}')

overall_rmsle = np.sqrt(mean_squared_log_error(np.expm1(y), np.expm1(oof_dnn)))
print(f"\nOverall OOF RMSLE: {overall_rmsle:.6f}")  

test_dnn /= n_folds


I0000 00:00:1746524711.814523      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1746524711.815272      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Epoch 1/25


I0000 00:00:1746524719.200451      61 service.cc:148] XLA service 0x7c32ec007390 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746524719.201786      61 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1746524719.201804      61 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1746524719.707670      61 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1746524723.420068      61 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2250/2250 - 16s - 7ms/step - loss: 0.0929 - root_mean_squared_error: 0.3048 - val_loss: 0.0065 - val_root_mean_squared_error: 0.0805 - learning_rate: 0.0500
Epoch 2/25
2250/2250 - 6s - 3ms/step - loss: 0.0188 - root_mean_squared_error: 0.1372 - val_loss: 0.0061 - val_root_mean_squared_error: 0.0778 - learning_rate: 0.0500
Epoch 3/25
2250/2250 - 6s - 3ms/step - loss: 0.0152 - root_mean_squared_error: 0.1231 - val_loss: 0.0065 - val_root_mean_squared_error: 0.0806 - learning_rate: 0.0500
Epoch 4/25
2250/2250 - 6s - 3ms/step - loss: 0.0157 - root_mean_squared_error: 0.1254 - val_loss: 0.0811 - val_root_mean_squared_error: 0.2848 - learning_rate: 0.0500
Epoch 5/25
2250/2250 - 6s - 3ms/step - loss: 0.0602 - root_mean_squared_error: 0.2453 - val_loss: 0.0076 - val_root_mean_squared_error: 0.0870 - learning_rate: 0.0500
Epoch 6/25
2250/2250 - 6s - 2ms/step - loss: 0.0179 - root_mean_squared_error: 0.1338 - val_loss: 0.0197 - val_root_mean_squared_error: 0.1402 - learning_rate: 0.0500
Epoch 7/

In [19]:
test_dnn = np.expm1(test_dnn)
test_dnn = np.clip(test_dnn, 1, 314)

sub = pd.read_csv('/kaggle/input/playground-series-s5e5/sample_submission.csv')
sub['Calories'] = test_dnn
sub.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")
print(sub.head(5))

Your submission was successfully saved!
       id    Calories
0  750000   27.998877
1  750001  105.904615
2  750002   86.594670
3  750003  123.738918
4  750004   75.778031
